## Scrapping the dataset from AUTO.AM


### Importing and Configuration

In [ ]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [99]:
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options = options)
#driver.maximize_window()

In [96]:
driver.get('https://auto.am/lang/en')
driver.set_window_size(1000, 1400)


### Navigating to main page where the links of cars are present

In [89]:
search_button = driver.find_element(By.ID, value='search-btn')
search_button.click()

In [82]:
maks = driver.find_element(By.XPATH, value="//*[contains(text(), 'Custom Cleared')]")
maks.click()

ElementClickInterceptedException: Message: element click intercepted: Element <label for="ch-custcleared">...</label> is not clickable at point (106, 683). Other element would receive the click: <div id="research" class="fixed">...</div>
  (Session info: chrome=124.0.6367.119)
Stacktrace:
0   chromedriver                        0x000000010055a8cc chromedriver + 4368588
1   chromedriver                        0x0000000100552d60 chromedriver + 4336992
2   chromedriver                        0x0000000100176c04 chromedriver + 289796
3   chromedriver                        0x00000001001be88c chromedriver + 583820
4   chromedriver                        0x00000001001bce20 chromedriver + 577056
5   chromedriver                        0x00000001001bafac chromedriver + 569260
6   chromedriver                        0x00000001001ba13c chromedriver + 565564
7   chromedriver                        0x00000001001af940 chromedriver + 522560
8   chromedriver                        0x00000001001af368 c

### Extracting the car links from main page

In [7]:
car_links = []

In [8]:
def link_exctractor():
    xpath = "//*[@id = 'search-result']//*[@class ='card-content' ]/a[@href]"
    links = driver.find_elements(By.XPATH, xpath)
    for link in links:
        car_links.append(link.get_attribute('href'))


In [9]:
current_page = 1
while True:
    if current_page == 345:
        break
    time.sleep(1)
    link_exctractor()
    next_page = driver.find_element(By.XPATH, "(//*[@class = 'waves-effect clickable nav'])[last()]")
    next_page.click()
    current_page += 1

KeyboardInterrupt: 

In [57]:
car_links = list(set(car_links))

In [58]:
len(car_links)

10000

### Implementing the code that will iterate over the collected links and gather information about cars

In [59]:
car_list = []

In [60]:
failed_links = []

In [77]:

def final(link_list, current_stage = 0):
    for i in range(current_stage,len(link_list)):
        try:
            driver.get(link_list[i])
            time.sleep(1)

            dicto = dict()

            car_xpath = "//*[@id='sec-blue']/div[1]/div/div[1]/h1/a[3]"
            car = driver.find_element(By.XPATH,
                                      car_xpath).get_attribute('innerHTML')
            
            model_xpath = "//*[@id='sec-blue']/div[1]/div/div[1]/h1/a[4]"
            model = driver.find_element(By.XPATH,
                                        model_xpath).get_attribute('innerHTML')
            
            year_xpath = "//*[@id='sec-blue']/div[1]/div/div[1]/h1/a[2]"
            year = driver.find_element(By.XPATH,
                                             year_xpath).get_attribute('innerHTML')
            
            price_xpath = "//*[@id='pricedown']//span"
            price = driver.find_element(By.XPATH,
                                        price_xpath).get_attribute('innerHTML')
            
            region_xpath = "//*[@id='sec-blue']/div[1]/div/div[1]/div/span[2]"
            region = driver.find_element(By.XPATH, 
                                         region_xpath).get_attribute('innerHTML')

            dicto['car'] = car
            dicto['model'] = model
            dicto['year'] = year
            dicto['price'] = price
            dicto['region'] = region
            
            trs = driver.find_elements(By.XPATH, "//*[@id='tab1']//tbody/tr")
            for item in trs:
                tds = item.find_elements(By.XPATH, "td")
                dicto[tds[0].text] = tds[1].text
            car_list.append(dicto)
            dicto['link'] = f'{link_list[i]}'
            current_stage += 1
        except NoSuchElementException as e:
            print(f'Error for the {i} th element')
            failed_links.append(link_list[i])
            current_stage += 1
            continue
    
        except:
            print(f'Error occured, we are on the {current_stage}th item')
            return current_stage
        

    

### manually launching the code with batches and then merging the dataset scrapped from each batch, since scrapping requires long process.

In [98]:
x = final(car_links, 9235)

Error occured, we are on the 9235th item


In [100]:
len(car_list)

8748

In [92]:
final_dataset = pd.DataFrame(car_list)

In [93]:
final_dataset

,car,model,year,price,region,Mileage,Body Style,Engine,Gearbox,Hand Drive,...,Engine Volume,Horsepower,Engine Cylinders,Drive train,Door Count,Wheels,Version,Battery capacity in kW,Number of electric motors,Range
0,Nissan,Rogue,2018,15 900 $,"Armenia, Yerevan",36000 miles,SUV / Truck,Gasoline,CVT Variable,Left,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mazda,3,2015,10 000 $,"Armenia, Yerevan",137000 km,Sedan,Gasoline,Automatic,Left,...,2.0,155 hp,4,FWD,4,"16""",NaN,NaN,NaN,NaN
2,Mercedes-Benz,E 350,2011,15 000 $,"Armenia, Yerevan",230000 km,Sedan,Gasoline,Automatic,Left,...,3.5,272 hp,6,RWD,4,"18""",NaN,NaN,NaN,NaN
3,Mercedes-Benz,E 230,1996,4 400 $,"Armenia, Kotayk",330000 km,Sedan,Gasoline,Automatic,Left,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mercedes-Benz,GL 450,2007,16 500 $,"Armenia, Yerevan",270000 km,SUV / Truck,Gasoline and gas,Automatic,Left,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8484,Lexus,RX 350,2010,15 500 $,"Armenia, Yerevan",91000 miles,SUV / Truck,Gasoline,Automatic,Left,...,3.5,275 hp,6,NaN,NaN,"19""",Lexus RX350 2010թ 3.5լ Sport,NaN,NaN,NaN
8485,Nissan,Altima,2013,9 600 $,"Armenia, Yerevan",146000 miles,Sedan,Gasoline,CVT Variable,Left,...,2.5,173 hp,NaN,FWD,4,"17""",L33,NaN,NaN,NaN
8486,Mazda,CX5,2020,24 000 $,Armenia,31000 km,SUV / Truck,Gasoline,Automatic,Left,...,NaN,NaN,NaN,NaN,NaN,NaN,4X4 Grand Touring,NaN,NaN,NaN
8487,Volkswagen,Jetta,2013,11 500 $,"Armenia, Yerevan",129900 km,Sedan,Gasoline,Automatic,Left,...,2.0,NaN,NaN,NaN,NaN,"16""",NaN,NaN,NaN,NaN


In [94]:
final_dataset.to_csv('final_dataset.csv')